In [30]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt # Open Korean Text. 한국어 형태소 분석기
import gensim
from gensim import corpora, models

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [34]:
# 폰트 설정
font_path = '/Users/heungrokoh/Library/Fonts/D2Coding-Ver1.3.2-20180524.ttf'
plt.rcParams['font.family'] = 'D2Coding'
fm.fontManager.addfont(font_path)
fm.FontProperties(fname=font_path)

In [3]:
work_dir = '/Users/heungrokoh/python_workspace/work/voucher_test'

In [4]:
# Load the data
df = pd.read_excel(f"{work_dir}/data/채널별샘플데이터_데이터바우처사업_파프리카스토리.xlsx", sheet_name='5) 네이버쇼핑리뷰')

In [5]:
df

,"스탠바이미, 스탠바이미고 네이버 쇼핑 리뷰- 쇼핑몰 리뷰 부분",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,스탠바이미 - https://search.shopping.naver.com/cata...,스탠바이미고- https://search.shopping.naver.com/cat...,NaN,NaN,NaN,NaN,NaN
1,제목,내용,구매처,평점,작성날짜,사용자,상품
2,조아용 조아용 조아용 ㅎㅎ,조아용 조아용 조아용 ㅎㅎ,오아시스486,5,24.05.15.,rorn****,스탠바이미
3,너무 만족하며 사용중입니다,너무 만족하며 사용중입니다,Ader,5,24.05.15.,arde****,스탠바이미
4,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,Ader,5,24.05.15.,jeon****,스탠바이미
...,...,...,...,...,...,...,...
997,선물용으로 구매했는데 빠른 배송 감사,선물용으로 구매했는데 빠른 배송 감사합니다.,LG전자,5,23.09.06.,c*0*****,스탠바이미고
998,사무실 휴대용으로 구매했습니다. 만족한것 같습니다,사무실 휴대용으로 구매했습니다. 만족한것 같습니다,LG전자인증점 신영플러스,5,23.09.06.,fore****,스탠바이미고
999,물건 잘 받았습니다 배송도 좋게 보내 주시고 제품도 좋고 조금 아쉬운 게 충전 충정...,물건 잘 받았습니다 배송도 좋게 보내 주시고 제품도 좋고 조금 아쉬운 게 충전 충정...,알렛츠ALLETS,5,23.09.05.,gilg****,스탠바이미고
1000,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,오늘의집,5,23.09.05.,1*6*****,스탠바이미고


In [6]:
df.columns = df.iloc[1]
df = df[2:].reset_index(drop=True)
df.columns.name = None
df

,제목,내용,구매처,평점,작성날짜,사용자,상품
0,조아용 조아용 조아용 ㅎㅎ,조아용 조아용 조아용 ㅎㅎ,오아시스486,5,24.05.15.,rorn****,스탠바이미
1,너무 만족하며 사용중입니다,너무 만족하며 사용중입니다,Ader,5,24.05.15.,arde****,스탠바이미
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,Ader,5,24.05.15.,jeon****,스탠바이미
3,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,Ader,5,24.05.15.,pjw9****,스탠바이미
4,좋아요,살까말까 했는데 좋아요,11번가,4,24.05.15.,m0******,스탠바이미
...,...,...,...,...,...,...,...
995,선물용으로 구매했는데 빠른 배송 감사,선물용으로 구매했는데 빠른 배송 감사합니다.,LG전자,5,23.09.06.,c*0*****,스탠바이미고
996,사무실 휴대용으로 구매했습니다. 만족한것 같습니다,사무실 휴대용으로 구매했습니다. 만족한것 같습니다,LG전자인증점 신영플러스,5,23.09.06.,fore****,스탠바이미고
997,물건 잘 받았습니다 배송도 좋게 보내 주시고 제품도 좋고 조금 아쉬운 게 충전 충정...,물건 잘 받았습니다 배송도 좋게 보내 주시고 제품도 좋고 조금 아쉬운 게 충전 충정...,알렛츠ALLETS,5,23.09.05.,gilg****,스탠바이미고
998,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,오늘의집,5,23.09.05.,1*6*****,스탠바이미고


In [7]:
txt_s5= df['내용']
print(txt_s5)

0                                         조아용 조아용 조아용 ㅎㅎ
1                                         너무 만족하며 사용중입니다
2          4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요
3                           여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요
4                                           살까말까 했는데 좋아요
                             ...                        
995                             선물용으로 구매했는데 빠른 배송 감사합니다.
996                          사무실 휴대용으로 구매했습니다. 만족한것 같습니다
997    물건 잘 받았습니다 배송도 좋게 보내 주시고 제품도 좋고 조금 아쉬운 게 충전 충정...
998    일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...
999    침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...
Name: 내용, Length: 1000, dtype: object


## 데이터 전처리

In [8]:
# 결측치 제거
txt_s5 = txt_s5.dropna()

In [9]:
# 특수문자 제거
txt_s5 = txt_s5.apply(lambda x: re.sub(r'[^가-힣]\s', " " , x)) # 한글과 공백을 제외한 모든 문자를 제거 (^: not, \s: 공백)


In [10]:
# 불용어 사전
## 불용어 추가 시, txt파일 내 추가하거나 append로 추가
f = open("./data/korean_stopwords.txt", 'r')
lines = f.readlines()
stopwords = []
for line in lines:
    line = line.replace('\n', '')
    stopwords.append(line)
f.close()

In [11]:
stopwords[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

In [12]:
# 형태소 분석 및 불용어 제거
okt = Okt()
txt_s5_morphs = []
for i in range(len(txt_s5)):
    words = okt.morphs(txt_s5[i], stem=True) # stem=True: 어간 추출(조아요 -> 좋다)
    words = [word for word in words if not word in stopwords] # 불용어 제거
    morphs = okt.pos(txt_s5[i], stem=True) # 품사 태깅
    words = [word for word, tag in morphs if tag in ['Noun', 'Adjective']] # 명사, 형용사만 추출
    txt_s5_morphs.append(words)
txt_s5_morphs[:10]

[['좋다', '좋다', '좋다'],
 ['만족하다', '사용', '이다'],
 ['설치', '배송', '연락', '원하다', '날', '안착', '완료', '제품', '설치', '서비스', '모두', '만족'],
 ['여자친구', '만족하다', '생일', '때', '걸', '그렇다'],
 ['살', '까맣다', '좋다'],
 ['빠르다', '배송', '직잡', '배송', '설치'],
 ['자취',
  '티비',
  '노트북',
  '패드',
  '유튜브',
  '시청',
  '확실하다',
  '더',
  '화면',
  '보',
  '좋다',
  '설거지',
  '거나',
  '가구',
  '배치',
  '때',
  '움직',
  '이동',
  '수',
  '있다',
  '정말',
  '좋다',
  '삶',
  '질',
  '향상'],
 ['핸드폰',
  '를',
  '항상',
  '아쉽다',
  '고민',
  '고민',
  '문후',
  '다음',
  '날',
  '오전',
  '배송',
  '설치',
  '생각',
  '대로',
  '디자인',
  '품질',
  '맘'],
 ['주문', '바로', '설치다', '문자', '토요일', '기사', '설치'],
 ['예쁘다', '배송', '고맙다']]

In [ ]:
# 불용어 제거
txt_s5_morphs = [[word for word in words if not word in stopwords] for words in txt_s5_morphs]